<a href="https://colab.research.google.com/github/krishna-k70/1st-project/blob/main/AI4SEE_pvt_Ltd(Assignment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary libraries
!pip install opencv-python-headless


In [5]:
# Download YOLO files
!wget -q https://pjreddie.com/media/files/yolov3.weights
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


In [6]:
# Import necessary libraries
import cv2
import numpy as np
import os
from google.colab import files

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Function to detect objects
def detect_objects(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return [(class_ids[i], confidences[i], boxes[i]) for i in indexes.flatten()]


In [7]:
# Create output directory if not exists
output_dir = "cropped_objects"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Upload video
uploaded = files.upload()


Saving video.mp4 to video (2).mp4


In [8]:
# Open video
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)

# Get video frame rate
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Frames per second: {fps}")

# Set duration and calculate total frames
duration = 7  # in seconds
total_frames = int(fps * duration)
print(f"Total frames expected for 7 seconds: {total_frames}")

frame_count = 0

while frame_count < total_frames:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    objects = detect_objects(frame)

    for i, (class_id, confidence, box) in enumerate(objects):
        x, y, w, h = box
        # Check if the coordinates are valid
        if x < 0 or y < 0 or x + w > frame.shape[1] or y + h > frame.shape[0]:
            print(f"Invalid coordinates for cropping: x={x}, y={y}, w={w}, h={h}")
            continue

        cropped = frame[y:y+h, x:x+w]

        if cropped.size == 0:
            print(f"Empty cropped image at frame {frame_count}, object {i}, coordinates: {x}, {y}, {w}, {h}")
            continue

        obj_name = classes[class_id]
        save_path = f"{output_dir}/frame_{frame_count}_object_{i}_{obj_name}.jpg"
        cv2.imwrite(save_path, cropped)
        print(f"Saved cropped image to {save_path}")

    print(f"Processed frame {frame_count}")

cap.release()
cv2.destroyAllWindows()

# Print total frames processed
print(f"Total frames processed: {frame_count}")


Streaming output truncated to the last 5000 lines.
Saved cropped image to cropped_objects/frame_144_object_19_person.jpg
Processed frame 144
Invalid coordinates for cropping: x=921, y=316, w=254, h=408
Invalid coordinates for cropping: x=734, y=233, w=247, h=504
Invalid coordinates for cropping: x=451, y=310, w=253, h=424
Saved cropped image to cropped_objects/frame_145_object_3_person.jpg
Saved cropped image to cropped_objects/frame_145_object_4_person.jpg
Saved cropped image to cropped_objects/frame_145_object_5_person.jpg
Saved cropped image to cropped_objects/frame_145_object_6_person.jpg
Saved cropped image to cropped_objects/frame_145_object_7_person.jpg
Saved cropped image to cropped_objects/frame_145_object_8_person.jpg
Saved cropped image to cropped_objects/frame_145_object_9_person.jpg
Invalid coordinates for cropping: x=-1, y=424, w=126, h=282
Saved cropped image to cropped_objects/frame_145_object_11_person.jpg
Saved cropped image to cropped_objects/frame_145_object_12_pers

In [9]:
# Zip the output directory for easy download if needed
!zip -r cropped_objects.zip cropped_objects


Streaming output truncated to the last 5000 lines.
updating: cropped_objects/frame_156_object_8_person.jpg (deflated 2%)
updating: cropped_objects/frame_77_object_12_person.jpg (deflated 3%)
updating: cropped_objects/frame_366_object_5_person.jpg (deflated 1%)
updating: cropped_objects/frame_387_object_3_person.jpg (deflated 1%)
updating: cropped_objects/frame_270_object_11_person.jpg (deflated 4%)
updating: cropped_objects/frame_70_object_17_person.jpg (deflated 3%)
updating: cropped_objects/frame_230_object_3_person.jpg (deflated 1%)
updating: cropped_objects/frame_272_object_9_person.jpg (deflated 1%)
updating: cropped_objects/frame_400_object_5_person.jpg (deflated 5%)
updating: cropped_objects/frame_132_object_5_person.jpg (deflated 1%)
updating: cropped_objects/frame_306_object_15_person.jpg (deflated 3%)
updating: cropped_objects/frame_318_object_4_person.jpg (deflated 0%)
updating: cropped_objects/frame_305_object_8_person.jpg (deflated 2%)
updating: cropped_objects/frame_61_ob